In [1]:
import os
import random
import numpy as np
import pandas as pd
import math
from tqdm import tnrange, tqdm_notebook


from glob import glob
from IPython.display import display

In [2]:
#Load Validation dataset & pCTRs
Path = 'C://Datasets//Multiagent//calc'

#Importing pCTR data.
os.chdir(Path)
Datasets_CTR = {}

for Filename in glob('*validation_pCTR.csv'):
    Datasets_CTR[Filename[:-4]] = pd.read_csv(Filename, sep = ',')

#Importing Validation data.
os.chdir('c://Datasets//Multiagent//rtb')
Datasets_features = {}
for Filename in glob('Validation.csv'):
    Datasets_features[Filename[:-4].title()] = pd.read_csv(Filename, sep = ',')
    
#Defining variables.
Validation = Datasets_features['Validation']

In [3]:
# Define Bid Strategies
def linear_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*pCTR/Init_CTR)

def nlogn_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*(pCTR/Init_CTR)*np.log(pCTR/Init_CTR))

def exp_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*np.e**(pCTR/Init_CTR))


In [16]:
class Agent:
    
    initPCR = 0.0007375623256
    initBudget = 6250
        
    def __init__(self, id, pCTR_strategy, bid_strategy, params):
        self.budget = self.initBudget
        self.id = id
        self.pCTR_strategy = pCTR_strategy
        self.bid_strategy = bid_strategy
        self.params = params        
        self.pCTRs = list(Datasets_CTR[self.pCTR_strategy +'_validation_pCTR']['pCTR'].values)
        self.bids = tuple()
        self.initBids()
            
    def initBids(self):
        bids = []
        for pCTR in self.pCTRs:
            if(self.bid_strategy == "Linear"):
                bids.append(linear_bidding(self.params[0],pCTR,self.initPCR))
            elif(self.bid_strategy == "NLogN"):
                bids.append(nlogn_bidding(self.params[0],pCTR,self.initPCR)) 
            elif(self.bid_strategy == "Exp"):
                bids.append(exp_bidding(self.params[0],pCTR,self.initPCR))
            else:
                raise("Bid strategy not found for agent ", self.id)
        self.bids = bids    
        return
        
     
    # Update budget based on bid, returns True if budget allows
    def updateBudget(self,bid,overbid=False):
        if(not overbid):
            if (self.budget - bid) < 0:
                print("ERROR: not enough budget for bid")
                return False
            else:
                self.budget = self.budget - bid
                return True
        else:
            self.budget = self.budget - bid
            return

        
    def canBid(self,bid):
        if(self.budget>bid):
            return True
        return False
    
    #Ensure that bid is within budget
    def getMaxBid(self,idx,overbid=False):
        if (overbid and self.budget>0) or (self.budget > self.bids[idx]):
            return self.bids[idx]
        else:
            return self.budget
    
    def addBids(self,bids):
        self.bids = bids
        return
    

In [17]:


Agent1 =  Agent(1,  "LR", "Linear", [1])
Agent2 =  Agent(2,  "LR", "Linear", [100])
Agent3 =  Agent(3,  "LR", "Exp", [30])
Agent4 =  Agent(4,  "LR", "Linear", [105])
Agent5 =  Agent(5,  "LR", "Exp", [29])
Agent6 =  Agent(6,  "LR", "Linear", [121])
Agent7 =  Agent(7,  "LR", "NLogN", [125])
Agent8 =  Agent(8,  "LR", "Linear", [95])
Agent9 =  Agent(9,  "LR", "Exp", [31])
Agent10 = Agent(10, "LR", "Linear", [110])
Agent11 = Agent(11, "LR", "Linear", [105])
Agent12 = Agent(12, "LGBM","Linear",[120])
Agent13 = Agent(13, "LGBM","Linear",[115])
Agent14 = Agent(14, "LGBM","Exp",[35])
Agent15 = Agent(15, "LGBM","Exp",[30])
Agent16 = Agent(16, "LGBM","NLogN",[10])
Agent17 = Agent(17, "LGBM","NLogN",[20])


Agents = [Agent1,Agent2,Agent3,Agent4,Agent5,Agent6,Agent7,Agent8,Agent9,Agent10,Agent11, Agent12, Agent13,Agent14,Agent15,Agent16,Agent17]

debug = True


#df = pd.DataFrame(AgentsBids, columns=["Agent","Bid"])

#AgentsBidsDf = pd.DataFrame(AgentsBids, columns=["Agent","Bid"])



def getBidsCurrentAuction(agents, idx, overbid = False, verbose=False):
    bidsCurrentAuction = []
    for agent in agents:        
        if(verbose):
            print('Bid[',agent.id,']:', agent.getMaxBid(idx,overbid))
        bidsCurrentAuction.append([agent, agent.getMaxBid(idx,overbid)])
    return pd.DataFrame(bidsCurrentAuction, columns=["Agent","Bid"])

def secondPriceWinner(AgentsBidsDf, floorPrice, idx=0, overbid=False,verbose=False):
    #if(idx> 288398 ):
    #    verbose=True
    bestBids= AgentsBidsDf.sort_values(by="Bid", ascending=False)
    if(verbose):
        print(bestBids)
    highestBidAgent = bestBids.iloc[0,0]
    curr_bid = bestBids.iloc[0,1]
    secondPriceOthers = bestBids.iloc[1,1]
    #print(curr_bid, floorPrice)
    if(curr_bid >= floorPrice):
        actualPayPrice = max(floorPrice,secondPriceOthers)
        highestBidAgent.updateBudget(actualPayPrice/1000, overbid)
        if (verbose):
            print(idx,"\tWon: ", highestBidAgent.id, " Paid:" , '{:.3f}'.format(floorPrice), " Upd budget:", '{:.3f}'.format(highestBidAgent.budget))
        return highestBidAgent
    else:        
        if(verbose):
            print(idx, "\tNo winner, Floor:",floorPrice)
        return None
    
    

def individualAuctions(verbose=False):
    overbid = False
    results_1 = {}
    for idx in tnrange(len(Validation), desc='Simulating auction'):
    #for idx in tnrange(288602, desc='Simulating auction'):    
        #curr_bid = Validation.iloc[idx]
        #if(verbose):
        #    print (Validation.iloc[idx])
        df = getBidsCurrentAuction(Agents,idx,overbid,verbose=False)
        floorPrice = Validation['payprice'][idx]
        winner = secondPriceWinner(df,floorPrice,idx,overbid,verbose=verbose)
        if (winner != None):
            results_1[idx]=(idx,winner.id,winner.budget)            
        else:
            results_1[idx]=(idx,np.NaN,np.NaN)
        
    results_df=pd.DataFrame.from_dict(results_1).transpose()
        
    return results_df
        


C:\Users\mau_c\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in double_scalars
  if __name__ == '__main__':


In [18]:
df = individualAuctions(False)
df.rename(columns={0:'Ad', 1:'Winner', 2:'Budget'},inplace=True)
df.head()

,Ad,Winner,Budget
0,0.0,6.0,6249.906045
1,1.0,NaN,NaN
2,2.0,NaN,NaN
3,3.0,6.0,6249.860021
4,4.0,9.0,6249.959755


In [7]:
#df.rename(columns={0:'Ad', 1:'Winner', 2:'Budget'}, inplace=True)
#df.head()

In [19]:
results ={}
for idx in range(len(Agents)):    
    agent = Agents[idx]    
    a_idx_won = df[df["Winner"] == agent.id]['Ad']    
    a_won = Validation.iloc[a_idx_won]
    a_clicks = a_won[a_won['click'] == 1]['click'].sum()
    a_imps = len(a_won)
    a_spent = -1 * (agent.budget-Agent.initBudget)    
    results[idx] = ('Agent'+str(agent.id),agent.pCTR_strategy,agent.bid_strategy,agent.params,a_clicks,a_imps,round(a_spent,3))
#results
results_df = pd.DataFrame.from_dict(results).transpose()
results_df.rename(columns={0:'Agent', 1:'pCTR', 2:'Bidding',3:'BB', 4:'Clicks', 5:'Imps' , 6:'Spent' }, inplace=True)

results_df.sort_values('Clicks', ascending=False)

,Agent,pCTR,Bidding,BB,Clicks,Imps,Spent
6,Agent7,LR,NLogN,[125],32,3853,5643.55
11,Agent12,LGBM,Linear,[120],24,58810,6194.32
14,Agent15,LGBM,Exp,[30],18,3329,5877.01
13,Agent14,LGBM,Exp,[35],15,38293,6154.23
9,Agent10,LR,Linear,[110],14,12529,2637.16
12,Agent13,LGBM,Linear,[115],14,5606,890.618
5,Agent6,LR,Linear,[121],13,55038,6204.79
4,Agent5,LR,Exp,[29],12,2594,5765.33
8,Agent9,LR,Exp,[31],8,11035,6030.85
2,Agent3,LR,Exp,[30],6,3199,5867.33


In [14]:
results_df['Clicks'].sum()


160

In [ ]:
#ind = ()
#print(type(ind))
#print(ind[0])
#Validation.iloc[df[df.iloc[:,1] == 1][0]].count()


In [ ]:
ind_temp = 1688
df.iloc[ind_temp:ind_temp+60]

## Error in 288402

In [ ]:
bestBids= df.sort_values(by="Bid", ascending=False)
bestBids

In [ ]:
print("best bid by: ", bestBids.iloc[0,0].id, " second " , bestBids.iloc[1,0].id)
print("Win price: ", bestBids.iloc[0,1], " Price to pay " , bestBids.iloc[1,1])
